In [12]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding

In [61]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

uniform_data = np.random.rand(10, 12)
# ax = sns.heatmap(uniform_data, linewidth=0.5)
# plt.show()
print(uniform_data)

[[0.47839219 0.17776147 0.79758183 0.88493546 0.16646949 0.99795809
  0.91495392 0.29381124 0.56667666 0.03200929 0.87742416 0.4183641 ]
 [0.416652   0.28824879 0.87442369 0.06558953 0.53425249 0.73491354
  0.85112054 0.0633482  0.79794548 0.25467128 0.11157659 0.97817396]
 [0.36042139 0.31764001 0.05524164 0.78987019 0.28162053 0.39432902
  0.4784339  0.03517764 0.32156493 0.16587462 0.2024823  0.75313099]
 [0.8344614  0.97479294 0.89726535 0.9687031  0.94315161 0.25284866
  0.48990454 0.33870962 0.41629825 0.53747576 0.38295403 0.58089001]
 [0.80694232 0.60343967 0.05372172 0.84939195 0.06142543 0.79158843
  0.44429666 0.94663457 0.47153456 0.65066054 0.64947218 0.01801317]
 [0.95024838 0.42666904 0.41481192 0.62074665 0.89312897 0.23143024
  0.73934405 0.56186926 0.3865468  0.07548652 0.53482266 0.4558715 ]
 [0.0043266  0.9750888  0.35176436 0.17275985 0.39554458 0.25074902
  0.79826954 0.92075916 0.97151917 0.64137646 0.77148631 0.5008201 ]
 [0.6621237  0.89402716 0.65506875 0.3487

In [11]:
data_directory = r"D:\Courses\0\1 Programming\Music-Generation-using-deep-learning-main\data/"
# D:\Courses\0\1 Programming\Music-Generation-using-deep-learning-main
data_file = r"input.txt"
charIndex_json = "char_to_idx.json"
model_weights_directory = r'D:\Courses\0\1 Programming\Music-Generation-using-deep-learning-main\model/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [10]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    #remember, that here we haven't given return_sequences = True because here we will give only one character to generate the
    #sequence. In the end, we just have to get one output which is equivalent to getting output at the last time-stamp. So, here
    #in last layer there is no need of giving return sequences = True.
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [ ]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + "weights.{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    #above code is for ignoring the starting string of a generated sequence. This is because we are passing any arbitrary 
    #character to the model for generating music. Now, the model start generating sequence from that character itself which we 
    #have passed, so first few characters before "\n" contains meaningless word. Model start generating the music rhythm from
    #next line onwards. The correct sequence it start generating from next line onwards which we are considering.
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    #Now our data contains three newline characters after every tune. So, the model has leart that too. So, above code is used for
    #ignoring all the characters that model has generated after three new line characters. So, here we are considering only one
    #tune of music at a time and finally we are returning it..
    
    return seq2

In [59]:
epoch_num=102
initial_index=18
seq_length=500
with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
index_to_char = {i:ch for ch, i in char_to_index.items()}
unique_chars = len(index_to_char)

model = make_model(unique_chars)
model.load_weights(model_weights_directory + "weights.{}.h5".format(epoch_num))

sequence_index = [initial_index]

# for _ in range(seq_length):
batch = np.zeros((1, 1))
batch[0, 0] = sequence_index[-1]

predicted_probs = model.predict_on_batch(batch).ravel()
sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)

print("initial_index", initial_index)
i=initial_index
index_to_char = [ch for ch, i in char_to_index.items()]
print("equivalent character", index_to_char[i])
print("next predicted character", sample)
print(predicted_probs.shape)
# print(predicted_probs)

print(predicted_probs[sample])

np.around(predicted_probs,3)

#     sequence_index.append(sample[0])

#     seq = ''.join(index_to_char[c] for c in sequence_index)
# print(sequence_index)
    
#     cnt = 0
#     for i in seq:
#         cnt += 1
#         if i == "\n":
#             break
#     seq1 = seq[cnt:]
#     #above code is for ignoring the starting string of a generated sequence. This is because we are passing any arbitrary 
#     #character to the model for generating music. Now, the model start generating sequence from that character itself which we 
#     #have passed, so first few characters before "\n" contains meaningless word. Model start generating the music rhythm from
#     #next line onwards. The correct sequence it start generating from next line onwards which we are considering.
    
#     cnt = 0
#     for i in seq1:
#         cnt += 1
#         if i == "\n" and seq1[cnt] == "\n":
#             break
#     seq2 = seq1[:cnt]
#     #Now our data contains three newline characters after every tune. So, the model has leart that too. So, above code is used for
#     #ignoring all the characters that model has generated after three new line characters. So, here we are considering only one
#     #tune of music at a time and finally we are returning it..
    
#     return seq2

initial_index 18
equivalent character 3
next predicted character [5]
(87,)
[0.09706284]


array([0.003, 0.023, 0.001, 0.001, 0.   , 0.097, 0.005, 0.   , 0.015,
       0.001, 0.   , 0.001, 0.002, 0.001, 0.   , 0.002, 0.001, 0.   ,
       0.001, 0.   , 0.001, 0.001, 0.004, 0.001, 0.   , 0.581, 0.   ,
       0.   , 0.001, 0.   , 0.011, 0.   , 0.001, 0.003, 0.002, 0.001,
       0.003, 0.009, 0.006, 0.002, 0.001, 0.017, 0.003, 0.012, 0.044,
       0.   , 0.02 , 0.003, 0.093, 0.003, 0.004, 0.   , 0.002, 0.   ,
       0.002, 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.001, 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.004, 0.002], dtype=float32)

In [ ]:
ep = int(input("1. Enter the epoch number of the model you want to load. Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)